## Import all relevant libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
macro_master = pd.read_excel('Data_180724.xlsx', 'Macro')
price_master = pd.read_excel('Data_180724.xlsx','Prices')
treasuries_master = pd.read_excel('Data_180724.xlsx', 'Yield')
VIX_master = pd.read_excel('VIXCLS.xlsx','INDEX')

In [ ]:
macro_master.info()
price_master.info()
treasuries_master.info()
VIX_master.info()

### Data Visualisation to formulate regimes & technical strategy

In [ ]:
#f, axes = plt.subplots(nrows=1,ncols=2,figsize = (30,10))
f = plt.figure(figsize=(30,10))

#sb.lineplot(data = macro_master,x = macro_master['Date'],y = macro_master['REAL GDP YOY'], ax = axes[0])
sb.lineplot(data = macro_master,x = macro_master['Date'],y = macro_master['CPI YOY'])

##### Refer to notepad for breakdown of CPI and Real GDP analysis (Regime indicator: CPI)

In [ ]:
f = plt.figure(figsize=(30,10))

# Filter data for the desired time range (1970-1990)
#prices_filtered = price_master[(price_master['Date'] >= '1970-12-31') & (price_master['Date'] <= '1990-12-31')]

sb.lineplot(data = price_master,x = 'Date',y = 'INFLATION-ADJUSTED S&P 500')

In [ ]:
f1, axes1 = plt.subplots(nrows=1,ncols=3, figsize=(30,10))

i = 0
for item in ['Gold','USD Index Spot Rate','US 10YR Bonds']:
    if item == 'US 10YR Bonds':
        sb.lineplot(data = treasuries_master,x = 'Date', y = item,ax = axes1[i])
    else:
        sb.lineplot(data = price_master, x = 'Date', y = item,ax = axes1[i])
    i += 1

#### Long timeframe makes it difficult to identify opportunities. Sample time periods will be plotted to illustrate each strategy's methodology.

## Strategies Illustration

#### Based on notepad, 4 regimes are identified:
1. Overheating and Rising: CPI > 5% and increasing for >3 1M-periods
2. Overheating but Cooling: CPI > 5% and decreasing for >3 1M-periods
3. High and Rising: 3% < CPI < 5% sustained for >3 1M-periods 
4. Stablising: CPI trending between 1-3% for >3 consecutive periods
4. Cooling: CPI is trending below 1% (including below 0%)

Justification for cut-off values are previously covered.

CPI is a lagging indicator of economic growth. As CPI increases, financing costs increases eroding profits, reducing stock prices.
Monetary policy tightening is data-driven, hence stock price decline should follow after increasing (or high) CPI values. Vice versa for decreasing CPI.

CPI increase should increase risk premium as consumers and businesses flock to low-risk, store-of-value assets to avoid savings value erosion. This in turn increases market volatility in the stock market. Thus have to be more careful in volatile, high CPI periods.

#### How the strategy works for each regime:
1. Identify relationship between CPI and S&P500 index (or stock prices in general).
2. Identify macro indicator correlation with S&P500 index and hedging instruments.
3. Identify technical indicators entry and exit signals for S&P500 index, taking into account macro indicator position.
4. Select hedging instrument(s) and identify technical indicators for entry and exit signals.
4. Determine asset allocation for a particular regime (heavier on equity and less on cash for lower CPI regions, and vice versa.)

#### Strategy for overheating and rising CPI

In [ ]:
f, axes2 = plt.subplots(4,1,figsize=(30,20))

macro1_filtered = macro_master[(macro_master['Date'] >= '2000-01-01') & (macro_master['Date'] <= '2007-12-31')]
prices1_filtered = price_master[(price_master['Date'] >= '2000-01-01') & (price_master['Date'] <= '2007-12-31')]
treasuries1_filtered = treasuries_master[(treasuries_master['Date'] >= '2000-01-01') & (treasuries_master['Date'] <= '2007-12-31')]

ma_window = 10

sb.lineplot(data = macro1_filtered,x = 'Date',y = 'CPI YOY',ax = axes2[0])
sb.lineplot(data = prices1_filtered,x = 'Date',y = 'INFLATION-ADJUSTED S&P 500',ax = axes2[1])
sb.lineplot(data = prices1_filtered,x = 'Date',y = prices1_filtered['INFLATION-ADJUSTED S&P 500'].rolling(window=ma_window).mean(),ax = axes2[1])
sb.lineplot(data = prices1_filtered,x = 'Date',y = 'Gold',ax = axes2[2])
sb.lineplot(data = prices1_filtered,x = 'Date',y = prices1_filtered['Gold'].rolling(window=ma_window).mean(),ax = axes2[2])
sb.lineplot(data = prices1_filtered,x = 'Date',y = 'USD Index Spot Rate',ax = axes2[3])
sb.lineplot(data = prices1_filtered,x = 'Date',y = prices1_filtered['USD Index Spot Rate'].rolling(window=ma_window).mean(),ax = axes2[3])

#### Attempted Back-testing Algorithm

In [ ]:
prices_trading = price_master.copy()
prices_trading['10d-MA-S&P500'] = prices_trading['INFLATION-ADJUSTED S&P 500'].rolling(window=10).mean()
prices_trading['10d-MA-Gold'] = prices_trading['Gold'].rolling(window=10).mean()
prices_trading['10d-MA-USD Index'] = prices_trading['USD Index Spot Rate'].rolling(window=10).mean()
prices_trading['13d-MA-S&P500'] = prices_trading['INFLATION-ADJUSTED S&P 500'].rolling(window=13).mean()
prices_trading['13d-MA-Gold'] = prices_trading['Gold'].rolling(window=13).mean()
prices_trading['13d-MA-USD Index'] = prices_trading['USD Index Spot Rate'].rolling(window=13).mean()

In [ ]:
macro_trading = macro_master.copy()
macro_trading.drop(['REAL GDP YOY'],axis=1)

In [ ]:
# Calculate additional required columns
macro_trading['CPI_Change_3M'] = macro_trading['CPI YOY'].diff(3)
prices_trading['Gold_Change_3M'] = prices_trading['Gold'].diff(3)
prices_trading['USD_Change_3M'] = prices_trading['USD Index Spot Rate'].diff(3)

In [ ]:
#Signal Generation 
prices_trading['S&P500-Entry-Signal'] = (prices_trading['INFLATION-ADJUSTED S&P 500'] > prices_trading['10d-MA-S&P500'])
prices_trading['S&P500-Exit-Signal'] = (prices_trading['INFLATION-ADJUSTED S&P 500'] < prices_trading['13d-MA-S&P500'])

prices_trading['Gold_Buy_Signal'] = ((prices_trading['Gold_Change_3M'] < 0) & (prices_trading['Gold'].diff() > 0) & (prices_trading['Gold'] > prices_trading['10d-MA-Gold']))
prices_trading['Gold_Sell_Signal'] = ((prices_trading['Gold_Change_3M'] > 0) & (prices_trading['Gold'].diff() < 0) & (prices_trading['Gold'] > prices_trading['13d-MA-Gold']))

prices_trading['USD_Buy_Signal'] = ((prices_trading['USD_Change_3M'] < 0) & (prices_trading['USD Index Spot Rate'].diff() > 0) & (prices_trading['USD Index Spot Rate'] > prices_trading['10d-MA-Gold']))
prices_trading['USD_Sell_Signal'] = ((prices_trading['USD_Change_3M'] > 0) & (prices_trading['USD Index Spot Rate'].diff() < 0) & (prices_trading['USD Index Spot Rate'] > prices_trading['13d-MA-Gold']))

In [ ]:
#Initialize positions
prices_trading['S&P500_Position'] = 0
prices_trading['Gold_Position'] = 0
prices_trading['USD_Position'] = 0

In [ ]:
# Backtesting
for i in range(1, len(prices_trading)):
    if macro_trading['CPI YOY'][i] > 5 and macro_trading['CPI_Change_3M'][i] > 0:
        # Entry rules
        if data['SP500_Entry_Signal'][i]:
            data.at[i, 'SP500_Position'] = 1
            if data['Gold_Buy_Signal'][i]:
                data.at[i, 'Gold_Position'] = 1
            if data['USD_Buy_Signal'][i]:
                data.at[i, 'USD_Position'] = 1
        # Exit rules
        if data['SP500_Exit_Signal'][i]:
            data.at[i, 'SP500_Position'] = 0
            if data['Gold_Sell_Signal'][i]:
                data.at[i, 'Gold_Position'] = 0
            if data['USD_Sell_Signal'][i]:
                data.at[i, 'USD_Position'] = 0